In [81]:
import os

In [82]:
pwd

'/home/dhanu/MLOPS'

In [59]:
os.chdir("MLOPS")

In [83]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [84]:
os.chdir("src")

In [85]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [86]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [87]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [88]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load datasets
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features and target
        train_x = train_data.drop(columns=[self.config.target_column])
        test_x = test_data.drop(columns=[self.config.target_column])
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Train model
        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        model.fit(train_x, train_y)

        # Save model
        os.makedirs(self.config.root_dir, exist_ok=True)
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [89]:
os.chdir('../')

In [90]:
pwd

'/home/dhanu/MLOPS'

In [91]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-05-11 08:03:22,718: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-11 08:03:22,721: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-11 08:03:22,725: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-11 08:03:22,727: INFO: common: created directory at: artifacts]
[2025-05-11 08:03:22,729: INFO: common: created directory at: artifacts/model_trainer]
